# CARE Prediction Notebook

This notebook uses a trained CARE model to restore/denoise images.

## Setup

Make sure you have the required packages installed:
```bash
pip install tensorflow csbdeep tifffile numpy matplotlib aicsimageio aicspylibczi
```

In [ ]:
import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))

from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
%matplotlib inline

from aicsimageio import AICSImage
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

### Load Input Image

**Note:** Update the `path` below to match your input image location.

In [ ]:
# TODO: Update this path to your input image
path = r'path/to/your/data/fixed/CARE/raw_data/b2-2a_2c_pos6-01-_raw_lifeact_T0.czi'

tif = False
czi = True

if tif:
    img = tiff.imread(path)
elif czi:
    img = AICSImage(path).data

print(img.shape)
img = img[0,0,...]
axes = 'ZYX'

### Load Trained Model

**Note:** Update the `basedir` below to match where your trained model is saved.

In [ ]:
# TODO: Update basedir to your model location
model = CARE(config=None, name='my_model', basedir='path/to/your/CARE/models')

### Run Prediction

This may take some time depending on your image size and hardware.

In [ ]:
%%time
restored = model.predict(img, axes, n_tiles=(25,11,32))

### Save Restored Image

**Note:** Update the output path below.

In [ ]:
print(restored.shape)
import tifffile as tiff
import numpy as np

min_val = restored.min()
max_val = restored.max()

restored_norm = (restored - min_val) / (max_val - min_val)
restored_8bit = (65535 * np.clip(restored_norm, 0, 1)).astype(np.uint16)

# TODO: Update this output path
tiff.imwrite('path/to/your/data/fixed/CARE/raw_data/for_training/outputs/timelapset0.tif', restored_8bit)

### Load and Verify Saved Image (Optional)

In [ ]:
from tifffile import imread
import numpy as np

# TODO: Update this path to verify your output
restored = imread(r'path/to/your/data/fixed/CARE/raw_data/for_training/outputs/whole_predict.tif')
print(restored.shape)